<a href="https://colab.research.google.com/github/fairuznawar122/Network-Intrusion-Detection-System/blob/main/NIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import pandas as pd

# Specify the path to your file
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/LDAP.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/MSSQL.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/NetBIOS.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/Portmap.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/Syn.csv'
file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/UDP.csv'
# file_path = '/content/drive/MyDrive/Project Files/NIDS Docs/DDoS Datasets/UDPLag.csv'

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_path, low_memory=False)

# Display the first few rows of the DataFrame
# print(df.head())


In [54]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['Unnamed: 0', 'Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Le

In [55]:
# Now you can access the 'Label' column
label_counts = df['Label'].value_counts()

# Display the counts
print(label_counts)


Label
Syn       606749
UDP       112475
BENIGN      4068
UDPLag      1873
Name: count, dtype: int64


# Feature Names Used

In [56]:
feature_names = ['Fwd Packet Length Max',
                 'Fwd Packet Length Min',
                 'Min Packet Length',
                 'Max Packet Length',
                 'Average Packet Size',
                 'Fwd Packets/s',
                 'Fwd Header Length',
                 'Fwd Header Length.1',
                 'min_seg_size_forward',
                 'Total Length of Fwd Packets',
                 'Fwd Packet Length Std',
                 'Flow IAT Min',
                 'Subflow Fwd Bytes',
                 'Destination Port',
                 'Protocol',
                 'Packet Length Std',
                 'Flow Duration',
                 'Fwd IAT Total',
                 'ACK Flag Count',
                 'Init_Win_bytes_forward',
                 'Flow IAT Mean',
                 'Flow IAT Max',
                 'Fwd IAT Mean',
                 'Fwd IAT Max',
                 'Label']
len(feature_names)

25

In [57]:
df= df[feature_names]
df.shape

(725165, 25)

# Resampling Data to Balance it

In [58]:
import pandas as pd
from sklearn.utils import resample

# Creating the initial dataset


# Determine the target number of samples (the number of 'BENIGN' samples)
target_size = df[df['Label'] == 'BENIGN'].shape[0]

# Resample the dataset
ldap_resampled = resample(df[df['Label'] == 'UDPLag'],
                          replace=True,     # Sample with replacement
                          n_samples=target_size, # Match the 'BENIGN' count
                          random_state=42)    # For reproducibility

netbios_resampled = resample(df[df['Label'] == 'UDP'],
                             replace=True,
                             n_samples=target_size,
                             random_state=42)
syn = resample(df[df['Label'] == 'Syn'],
                             replace=True,
                             n_samples=target_size,
                             random_state=42)

benign_resampled = df[df['Label'] == 'BENIGN']  # BENIGN already matches the target size

# Combine all resampled categories back into one DataFrame
resampled_df = pd.concat([syn,netbios_resampled,ldap_resampled, benign_resampled])

# Shuffle the DataFrame to mix the samples
resampled_df = resampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new distribution
print(resampled_df['Label'].value_counts())


Label
Syn       4068
BENIGN    4068
UDP       4068
UDPLag    4068
Name: count, dtype: int64


In [59]:
resampled_df = resampled_df.reset_index(drop=True)
resampled_df

,Fwd Packet Length Max,Fwd Packet Length Min,Min Packet Length,Max Packet Length,Average Packet Size,Fwd Packets/s,Fwd Header Length,Fwd Header Length.1,min_seg_size_forward,Total Length of Fwd Packets,...,Packet Length Std,Flow Duration,Fwd IAT Total,ACK Flag Count,Init_Win_bytes_forward,Flow IAT Mean,Flow IAT Max,Fwd IAT Mean,Fwd IAT Max,Label
0,6.0,6.0,6.0,6.0,9.000000,2.000000e+06,40,40,20,12.0,...,0.000000,1,1.0,1,5840,1.000000e+00,1.0,1.000000e+00,1.0,Syn
1,6.0,6.0,6.0,6.0,8.000000,7.500000e+05,60,60,20,18.0,...,0.000000,4,4.0,0,292,2.000000e+00,3.0,2.000000e+00,3.0,BENIGN
2,393.0,321.0,321.0,393.0,401.500000,2.770851e+01,1462,1462,8,2088.0,...,33.617597,216540,216540.0,0,-1,4.330800e+04,110045.0,4.330800e+04,110045.0,UDP
3,6.0,6.0,6.0,6.0,9.000000,2.000000e+06,40,40,20,12.0,...,0.000000,1,1.0,1,5840,1.000000e+00,1.0,1.000000e+00,1.0,Syn
4,6.0,6.0,6.0,6.0,9.000000,2.000000e+06,40,40,20,12.0,...,0.000000,1,1.0,1,5840,1.000000e+00,1.0,1.000000e+00,1.0,Syn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16267,31.0,31.0,6.0,31.0,24.666667,1.639344e+04,32,32,32,31.0,...,14.433757,61,0.0,0,59,3.050000e+01,59.0,0.000000e+00,0.0,BENIGN
16268,375.0,375.0,375.0,375.0,562.500000,2.000000e+06,16,16,8,750.0,...,0.000000,1,1.0,0,-1,1.000000e+00,1.0,1.000000e+00,1.0,UDP
16269,6.0,6.0,6.0,6.0,6.600000,1.306715e-01,160,160,20,48.0,...,0.000000,61222235,61222235.0,1,5840,6.802471e+06,30167788.0,8.746034e+06,30167788.0,Syn
16270,6.0,6.0,6.0,6.0,9.000000,6.666667e+05,40,40,20,12.0,...,0.000000,3,3.0,0,16249,3.000000e+00,3.0,3.000000e+00,3.0,BENIGN


In [61]:
# Replace 'NetBIOS' with 1 and 'BENIGN' with 0 in the 'Category' column
resampled_df['Label'] = resampled_df['Label'].replace({'UDP': 1,'UDPLag': 1,'Syn':1, 'BENIGN': 0})

# Display the updated DataFrame
print(resampled_df['Label'].value_counts())
resampled_df.shape

Label
1    12204
0     4068
Name: count, dtype: int64


(16272, 25)

# Creating the Main Dataset

In [62]:
resampled_df.to_csv('d6.csv', index=False)